<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/RAG%20Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RAG Demo
Demonstriert die Implementierung des RAG-Pattern mit der OpenAI API.

##Setup

In [ ]:
!pip install openai numpy scikit-learn

## API-Zugriff konfigurieren

In [ ]:
import openai
import os
import time
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# OpenAI API Key aus Colab Secret auslesen und OpenAI Client erstellen
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=api_key)

## Beispieltexte erstellen

In [ ]:
knowledge_base = [
    "Python ist eine interpretierte, hochgradig abstrahierte Programmiersprache.",
    "Die OpenAI API ermöglicht den Zugriff auf fortschrittliche KI-Modelle.",
    "Machine Learning ist ein Teilgebiet der künstlichen Intelligenz.",
    # Weitere Texte hinzufügen ...
]

## Embeddings erzeugen

In [ ]:
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    print(text)
    print(response.data[0].embedding)
    return response.data[0].embedding

knowledge_embeddings = [get_embedding(doc) for doc in knowledge_base]


## Benutzerabfrage erstellen / passende Texte finden

In [ ]:
def find_most_relevant_document(user_query):
    query_embedding = get_embedding(user_query)
    similarities = cosine_similarity(
        [query_embedding],
        knowledge_embeddings
    )
    most_relevant_index = np.argmax(similarities)
    return knowledge_base[most_relevant_index]

user_query = "Was ist maschinelles Lernen?"
relevant_document = find_most_relevant_document(user_query)
print(f"Relevantestes Dokument: {relevant_document}")

## Antwort generieren

In [ ]:
def generate_response(user_query, context):
    prompt = (
        f"Beantworte die Frage basierend auf dem unten stehenden Kontext. \
        Wenn die Frage nicht mit den bereitgestellten Informationen beantwortet werden kann, antworte mit 'Ich weiß es nicht'.\n\n"
        f"Kontext: {context}\n\n"
        f"Frage: {user_query}"
    )
    messages = [
        {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content

response = generate_response(user_query, relevant_document)
print(f"Antwort: {response}")